In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

In [3]:
dataset_type='MNIST'
dataset_type='P7'
print("Dataset type= "+dataset_type)


num_hidden_units = 128*4*2#//2  # Number of hidden units of the RNN cell
#print(n_classes,batch_size, num_input)
CELL_TYPE = 'LSTM'
CELL_TYPE = 'RNN'
CELL_TYPE = 'GRU'
CELL_TYPE = 'SRNN' # For stacked RNN
CELL_TYPE = 'SGRU' # For stacked GRU

RNN_LAYER=2

learning_rate = 0.001 # The optimization initial learning rate
epoch_number=20# Total number of training epochs

if dataset_type == 'MNIST':
    batch_size = 100      # Training batch size
    # Data Dimension
    num_input = 28          # MNIST data input (image shape: 28x28)
    timesteps = 28          # Timesteps
    n_classes = 10          # Number of classes, one class per digit
else :
    batch_size = 69 # 414/6 is 69
    batch_size = 23
    num_input = 224*3      # Dogs data input (image shape: 224 raws and 224x3 columns)
    timesteps = 224        # Timesteps : each raw from an image will feed a deployed timestep cell.
    n_classes = 3          # Number of classes, one class per digit

#display_freq = batch_size    # Frequency of displaying the training results


Dataset type= P7


In [4]:
def reset_graph():
    if 'session' in globals() and session:
        print("\n*** Closing session...")
        session.close()
    tf.reset_default_graph()
    print("\n*** Graph has been reset! Graph should be rebuilt now!\n")

def load_data():
    '''Defaut path to where MNIST dataset is located (after download) :
    ~/.keras/datasets/mnist.npz
    '''
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    #-----------------------------------------------------------
    # Make data separation between valid and train data.
    #-----------------------------------------------------------
    x_valid_num = x_test.shape[0]//2
    part = x_train.shape[0]-x_valid_num

    x_valid = x_train[part:,:]
    y_valid = y_train[part:]
    x_train = x_train[:part,:]
    y_train = y_train[:part]
    
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def load_data_deprecated(mode='train'):
    """
    Function to (download and) load the MNIST data
    :param mode: train or test
    :return: images and the corresponding labels
    """
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)
    if mode == 'train':
        x_train, y_train, x_valid, y_valid = mnist.train.images, mnist.train.labels, \
                                             mnist.validation.images, mnist.validation.labels
        return x_train, y_train, x_valid, y_valid
    elif mode == 'test':
        x_test, y_test = mnist.test.images, mnist.test.labels
    return x_test, y_test


def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch

# weight and bais wrappers
def weight_variable(shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('Weight',
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('Bias',
                           dtype=tf.float32,
                           initializer=initial)

def RNN(x, weights, biases, timesteps, num_hidden, cell_type='RNN', layer_number=2):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    print("RNN() : Original X shape= {} / Timesteps= {}".format(x.shape, timesteps))
    x = tf.unstack(x, timesteps, 1)
    print("RNN() : Unstacked : Number of input tensors= {} / Input size= {} / Timesteps= {}"\
          .format(len(x), x[0].shape, timesteps))
    print("RNN() : Weights shape, Bias shape = {}, {}".format(weights.shape, biases.shape))

    # Define a rnn cell with tensorflow
    if cell_type == 'RNN' :
            type_cell = rnn.BasicRNNCell(num_hidden)
    elif cell_type == 'LSTM' :
        type_cell = tf.nn.rnn_cell.LSTMCell(num_hidden, name='basic_lstm_cell',state_is_tuple=True)
    elif cell_type == 'GRU' :
        type_cell = tf.contrib.rnn.GRUCell(num_hidden)
    elif cell_type == 'SRNN' :
        list_num_hidden = [num_hidden for _ in range(0,layer_number)]
        list_type_cell = [tf.contrib.rnn.BasicLSTMCell(num_units=n) for n in list_num_hidden]
        type_cell = tf.contrib.rnn.MultiRNNCell(list_type_cell)
    elif cell_type == 'SGRU' :
        list_num_hidden = [num_hidden for _ in range(0,layer_number)]
        list_type_cell = [tf.contrib.rnn.GRUCell(num_units=n) for n in list_num_hidden]
        type_cell = tf.contrib.rnn.MultiRNNCell(list_type_cell)
    else : 
        print("\n*** ERROR : cell type= {} NOT YET SUPPORTED!".format(type_cell))
        return None

    # Get LSTM cell output
    # If no initial_state is provided, dtype must be specified
    # If no initial cell state is provided, they will be initialized to zero
    output, current_state = rnn.static_rnn(type_cell, x, dtype=tf.float32)

    print("\n*** RNN() : Weights shape= {} / Tensor Output= {}".format(weights.shape, output[-1]))
    
    # Linear activation, using rnn inner loop last output
    #return tf.matmul(current_state, weights) + biases
    return tf.matmul(output[-1], weights) + biases

In [5]:
#x_train, y_train, x_valid, y_valid = load_data_deprecated()
#x_test, y_test = load_data_deprecated(mode='test')

import p8_util
if True :
    if dataset_type == 'MNIST' :
        dataset_filename = None
        x_train, x_valid, x_test, y_train, y_valid, y_test, n_classes,tuple_dimension \
        = p8_util.load_dataset(dataset_filename, dataset_type=dataset_type)
        print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, n_classes,tuple_dimension)
    else :
        dataset_filename = './data/arr_keras_X_y_train_test.dump'
        x_train, x_valid, y_train, y_valid, n_classes,tuple_dimension \
        = p8_util.load_dataset(dataset_filename, dataset_type=dataset_type)
        number = x_train.shape[0]
        w = x_train.shape[1]
        h = x_train.shape[2]
        c = x_train.shape[3]
        print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, n_classes,tuple_dimension)
        x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],-1)
        x_valid = x_valid.reshape(x_valid.shape[0],x_valid.shape[1],-1)
        print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, n_classes,tuple_dimension)
        
        x_train = np.transpose(x_train, [0,2,1])
        x_valid = np.transpose(x_valid, [0,2,1])
        x_test  = x_valid.copy()
        y_test  = y_valid.copy()
                               
        print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, n_classes,tuple_dimension)
else :
    x_train, y_train, x_valid, y_valid= load_data_deprecated()
    print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3) 3 (224, 224, 3)
(414, 224, 672) (47, 224, 672) (414, 3) (47, 3) 3 (224, 224, 3)
(414, 672, 224) (47, 672, 224) (414, 3) (47, 3) 3 (224, 224, 3)


In [6]:
y_train.shape, n_classes

((414, 3), 3)

In [7]:
#print(y_train[10])

In [8]:
def init_weight_and_bias(num_hidden_units, n_classes, timesteps, num_input):
    # Placeholders for inputs (x) and outputs(y)

    # create weight matrix initialized randomely from N~(0, 0.01)
    W = weight_variable(shape=[num_hidden_units, n_classes])

    # create bias vector initialized as zero
    b = bias_variable(shape=[n_classes])
    
    return W,b



## y_pred = tf.nn.softmax(output_logits)

# Model predictions
##cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')

#### Tensorflow Direct Acyclic Graph is built

In [9]:
reset_graph()
# Graph structure is built : 

# Variables initialization : Weights, Bias, X and Y.
W, b = init_weight_and_bias(num_hidden_units, n_classes, timesteps, num_input)
#W = tf.placeholder(tf.float32, shape=[num_hidden_units, n_classes], name='Weight')
#b = tf.placeholder(tf.float32, shape=[n_classes], name='Bias')
X = tf.placeholder(tf.float32, shape=[None, timesteps, num_input], name='X')
Y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')

# Get logits from RNN model
output_logits = RNN(X, W, b, timesteps, num_hidden_units, cell_type=CELL_TYPE, layer_number=RNN_LAYER)
    

# Define the loss function, is inserted into graph; loss function uses y,output_logits.
# cross-e,tropy will apply softmax to logit, compute cross-entropy and will perform average between 
# all classes.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_logits), name='loss')

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)

# y_prediction is a vector of indexes containing largest value in output_logitstimesteps, num_inputtimesteps, num_inputtimesteps, num_inputtimesteps, num_input
y_prediction = tf.argmax(output_logits, 1)

# y_true is a vector of indexes containing largest value in y
y_true = tf.argmax(Y, 1)

# A new node is inserted into graph : correct_prediction
# Following will result as an array of boolean values; True if indexes matches, False otherwise.
correct_prediction = tf.equal(y_prediction, y_true, name='correct_pred')

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='train_accuracy')


*** Graph has been reset! Graph should be rebuilt now!

RNN() : Original X shape= (?, 224, 672) / Timesteps= 224
RNN() : Unstacked : Number of input tensors= 224 / Input size= (?, 672) / Timesteps= 224
RNN() : Weights shape, Bias shape = (1024, 3), (3,)

*** RNN() : Weights shape= (1024, 3) / Tensor Output= Tensor("rnn/rnn/multi_rnn_cell/cell_1/gru_cell/add_223:0", shape=(?, 1024), dtype=float32)


In [10]:
batch_number = x_train.shape[0]//batch_size
batch_valid = 20
freq_display_per_batch=batch_number//5
print("\nCell Type= {}-{}Layers / Hidden units= {} / Batch size= {} / Batchs number= {} / Epochs={}"\
      .format(CELL_TYPE, RNN_LAYER, num_hidden_unitsnum_hidden_unitsnum_hidden_unitsnum_hidden_units, batch_size, batch_number, epoch_number))
#x_valid, y_valid = randomize(x_valid, y_valid)

if True :
    init = tf.global_variables_initializer()
    with tf.Session() as session:
        session.run(init)
        for epoch in range(0, epoch_number):
            print("Epoch= {} / {}".format(epoch+1, epoch_number))
            batch_start = 0
            x_train, y_train = randomize(x_train, y_train)
            for batch_iter in range(0, batch_number) :
                batch_end   = (batch_iter+1)*batch_size
                x_batch = x_train[batch_start:batch_end]
                y_batch = y_train[batch_start:batch_end]

                # Reshape x_batch to be compliant with X definition shape that is [None, timesteps, num_input]
                x_batch = x_batch.reshape(batch_size, timesteps, num_input)
                feed_dict_ = {X: x_batch, Y: y_batch}

                session.run(optimizer, feed_dict=feed_dict_)
                
                if batch_iter%freq_display_per_batch == 0 :
                    loss_eval,accuracy_eval = session.run([loss,accuracy], feed_dict=feed_dict_)
                    print("Batch= {0} / Loss={1:.2f} / Accuracy={2:.001%}".format(batch_iter, loss_eval,accuracy_eval))
                batch_start = batch_end
                
            # Loss and accuracy evaluation over test dataset.
            # After each EPOCH, new data are selected for evaluation.
            x_valid, y_valid = randomize(x_valid, y_valid)

            feed_dict_valid = {X: x_valid[:batch_valid].reshape((-1, timesteps, num_input)), Y: y_valid[:batch_valid]}
            loss_valid, acc_valid = session.run([loss, accuracy], feed_dict=feed_dict_valid)
            print("*** Epoch= {0}\t Validation count= {1}\t Loss={2:.2f}\t Accuracy={3:.01%}"\
                  .format(epoch+1, batch_valid, loss_valid,acc_valid))
            print("")
        #dict_input_rnn = {W:W, B:b}
        #logits_eval = session.run(RNN, feed_dict=dict_input_rnn)


Cell Type= SGRU-2Layers / Batch size= 23 / Batchs number= 18 / Epochs=20
Epoch= 1 / 20
Batch= 0 / Loss=1.77 / Accuracy=43.5%
Batch= 3 / Loss=1.37 / Accuracy=39.1%
Batch= 6 / Loss=1.17 / Accuracy=47.8%
Batch= 9 / Loss=0.95 / Accuracy=47.8%
Batch= 12 / Loss=1.27 / Accuracy=26.1%
Batch= 15 / Loss=1.22 / Accuracy=30.4%
*** Epoch= 1	 Validation count= 20	 Loss=1.08	 Accuracy=40.0%

Epoch= 2 / 20
Batch= 0 / Loss=1.10 / Accuracy=34.8%
Batch= 3 / Loss=1.23 / Accuracy=13.0%
Batch= 6 / Loss=1.06 / Accuracy=39.1%
Batch= 9 / Loss=1.05 / Accuracy=52.2%
Batch= 12 / Loss=1.05 / Accuracy=43.5%
Batch= 15 / Loss=1.10 / Accuracy=30.4%
*** Epoch= 2	 Validation count= 20	 Loss=1.12	 Accuracy=35.0%

Epoch= 3 / 20
Batch= 0 / Loss=1.15 / Accuracy=30.4%
Batch= 3 / Loss=1.23 / Accuracy=39.1%
Batch= 6 / Loss=1.15 / Accuracy=21.7%
Batch= 9 / Loss=1.05 / Accuracy=43.5%
Batch= 12 / Loss=0.90 / Accuracy=65.2%
Batch= 15 / Loss=1.04 / Accuracy=47.8%
*** Epoch= 3	 Validation count= 20	 Loss=1.15	 Accuracy=40.0%

Epoch

In [ ]:
#-----------------------------------------------------------------------
# Hidden units = 128*4*2
# Cell Type= SRNN-3Layers / Batch size= 23 / Batchs number= 18 / Epochs=20
#-----------------------------------------------------------------------


In [11]:
x_test.shape, y_test.shape
session